In [ ]:
"""
Purpose: To Build a Decomposition table that has the following additional features
1) Synapses inside the neuron object
2) Synapse information for that cell
3) How much was filtered away with dendrite on axon
4) Axon features
5) Baylor cell type
6) Allen cell type
7) Paired Nucleus information

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-06-22 21:55:47,077 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-22 21:55:47,081 - settings - Setting database.user to celiib
INFO - 2021-06-22 21:55:47,082 - settings - Setting database.password to newceliipass
INFO - 2021-06-22 21:55:47,083 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-22 21:55:47,084 - settings - Setting database.user to celiib
INFO - 2021-06-22 21:55:47,084 - settings - Setting database.password to newceliipass
INFO - 2021-06-22 21:55:47,097 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/de

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-06-22 21:55:47,768 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-22 21:55:47,827 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-22 21:55:47,827 - settings - Setting database.user to celiib
INFO - 2021-06-22 21:55:47,828 - settings - Setting database.password to newceliipass
INFO - 2021-06-22 21:55:47,829 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-22 21:55:48,498 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-22 21:55:48,551 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-22 21:55:48,553 - settings - Setting database.user to celiib
INFO - 2021-06-22 21:55:48,554 - settings - Setting database.password to newceliipass
INFO - 2021-06-22 21:55:48,555 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-22 21:55:48,557 - settings - Setting database.user to celiib
INFO - 2021-06-22 21:55:48,558 - settings - Setting database.pa

In [3]:
#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-06-22 21:55:51,734 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-22 21:55:51,736 - settings - Setting database.user to celiib
INFO - 2021-06-22 21:55:51,738 - settings - Setting database.password to newceliipass
INFO - 2021-06-22 21:55:51,743 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-22 21:55:52,537 - settings - Setting enable_python_native_blobs to True


In [4]:
import validation_utils as vu
import neuron_visualizations as nviz
import neuron_utils as nru
import proofreading_utils as pru
import time
import system_utils as su

In [10]:
import axon_utils as au

# Picking a Neuron to play with

In [6]:
(minnie.DecompositionAxon() & "axon_length > 1000")

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,axon_version key by which to lookup the finer axon processing method,decomposition saved neuron object with high fidelity axon,axon_length length (in um) of the classified axon skeleton,run_time the amount of time to run (seconds)
864691134876192250,0,0.25,30.00,0,0,0,5,=BLOB=,1024.4009593520238,481.46
864691134884741114,0,0.25,30.00,0,0,0,5,=BLOB=,2977.706081286783,493.71
864691134884742394,0,0.25,30.00,0,0,0,5,=BLOB=,1900.071761948801,240.42
864691134884742650,0,0.25,30.00,0,0,0,5,=BLOB=,2701.510739187321,384.04
864691134884743418,0,0.25,30.00,0,0,0,5,=BLOB=,1137.019678449479,173.0
864691134884743930,0,0.25,30.00,0,0,0,5,=BLOB=,1823.9167193276587,276.89
864691134884744186,0,0.25,30.00,0,0,0,5,=BLOB=,2731.685997661405,556.51
864691134884745210,0,0.25,30.00,0,0,0,5,=BLOB=,2163.87452222339,370.85
864691134884749562,0,0.25,30.00,0,0,0,5,=BLOB=,4155.7074353507915,661.96
864691134884750842,0,0.25,30.00,0,0,0,5,=BLOB=,1055.0899549526155,254.74


In [64]:
segment_id = 864691134884753146
neuron_objs,split_idxs = du.decomposition_with_spine_recalculation(segment_id,
                                         ignore_DecompositionAxon=True)

**Using table __decomposition for table_to_neuron_objs**
Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Dont need spine recalculation
Number of Neurons found = 1


In [66]:
import neuron
neuron_obj = neuron.Neuron(neuron_objs[0])

Recieved another instance of Neuron class in init -- so just copying data


In [67]:
# nviz.visualize_neuron(neuron_obj,
#                      limb_branch_dict="all")

# Funciton Parameters

In [85]:
import synapse_utils as syu
import classification_utils as clu
import preprocessing_vp2 as pre

verbose = True
validation = False
inh_exc_class_to_use_for_axon = "Baylor"

# Getting the nucleus id

In [69]:
ver = 88
nucleus_ids,nucleus_centers = du.segment_to_nuclei(segment_id,
                                                       nuclei_version=ver)
nucleus_ids

array([81966])

In [70]:
# perform_nucleus_pairing = True
# if perform_nucleus_pairing:
winning_nucleus_id, nucleus_info = nru.pair_neuron_obj_to_nuclei(neuron_obj,
                         "S0",
                          nucleus_ids,
                          nucleus_centers,
                         nuclei_distance_threshold = 15000,
                          return_matching_info = True,
                         verbose=True)
# else:
#     winning_nucleus_id = 12345
#     nucleus_info = dict()
#     nucleus_info["nuclei_id"] = winning_nucleus_id
#     nucleus_info["nuclei_distance"] = 0
#     nucleus_info["n_nuclei_in_radius"] = 1
#     nucleus_info["n_nuclei_in_bbox"] = 1
    
if verbose:
    print(f"nucleus_info = {nucleus_info}")
    print(f"winning_nucleus_id = {winning_nucleus_id}")

soma_center = [382486 890345 803109]
nuclei_within_radius = [81966]
nuclei_within_radius_distance = [1033.86556186]

There were 1 nuclei found within the radius of 15000 nm
winning_nuclei = 81966
winning_nuclei_distance = 1033.8655618599548

 For Bounding Box Search:
inside_nuclei = [81966]


At End: using return_id_0_if_no_matches = True
winning_nuclei = 81966
winning_nuclei_distance = 1033.8655618599548
n_nuclei_in_radius = 1
n_nuclei_in_bbox = 1
nucleus_info = {'nuclei_id': 81966, 'nuclei_distance': 1033.87, 'n_nuclei_in_radius': 1, 'n_nuclei_in_bbox': 1}
winning_nucleus_id = 81966


# Find the Cell Types Associated with the matched nuceli

In [71]:
allen_cell_type_info = ctu.allen_nuclei_classification_info_from_nucleus_id(81966)
allen_cell_type_info

{'e_i': 'excitatory',
 'e_i_n_nuc': 1,
 'cell_type': '5P_IT',
 'cell_type_n_nuc': 1,
 'cell_type_e_i': 'excitatory'}

# Adding the Synapses

In [72]:
neuron_obj = syu.add_synapses_to_neuron_obj(neuron_obj,
                            validation = validation,
                            verbose  = True,
                            original_mesh = None,
                            plot_valid_error_synapses = False,
                            calculate_synapse_soma_distance = False,
                            add_valid_synapses = True,
                              add_error_synapses=False)


---Step 1: Computing synapse_dict---
# of beginning_direct_connections = 70 
# of presyn: 36
# of postsyn: 34

---Step 2: Computing mesh_label_dict---
-- Working on presyn--
Using original_mesh_method
# of distance_errored_syn_idx = 0
# of mesh_errored_syn_idx = 0
# of valid_syn_idx = 36
-- Working on postsyn--
Using original_mesh_method
# of distance_errored_syn_idx = 0
# of mesh_errored_syn_idx = 0
# of valid_syn_idx = 34

---Step 3: add_valid_synapses_to_neuron_obj---
Synapse dict: 0.0049
Original Mesh: 0.1624
Closest Branch: 1.6311
Closest Skeleton Branch and distance from endpoint: 1.0297
Closest Branch: 1.62
Closest Skeleton Branch and distance from endpoint: 0.881
Total time for valid synapse objects = 0.0009865760803222656


In [74]:
neuron_obj_syn = neuron.Neuron(neuron_obj)

Recieved another instance of Neuron class in init -- so just copying data


# Running the Cell and Axon Classification

In [98]:
axon_angles

{2: {0: 142.18791629917555}}

In [76]:
#1) Run the cell classification

if verbose:
    print(f"\n\n ------ Part C: Inhibitory Excitatory Classification ---- \n\n")

filter_time = time.time()

(inh_exc_class,
 spine_category,
 axon_angles,
 n_axons,
 n_apicals,
 neuron_spine_density,
 n_branches_processed,
 skeletal_length_processed,
 n_branches_in_search_radius,
 skeletal_length_in_search_radius
 ) = clu.inhibitory_excitatory_classifier(neuron_obj_syn,
                                    return_spine_classification=True,
                                    return_axon_angles=True,
                                     return_n_axons=True,
                                     return_n_apicals=True,
                                     return_spine_statistics=True,
                                         axon_limb_branch_dict_precomputed=None,
                                    axon_angles_precomputed=None,
                                         verbose=verbose)
if verbose:
    print(f"Total time for classification = {time.time() - filter_time}")
    



 ------ Part C: Inhibitory Excitatory Classification ---- 


spine_category = densely_spined
endnodes_to_test = []
endnodes_to_test = []
endnodes_to_test = [479]
Skeleton paths distances = [24999.22664262]
Filtered indexes = [0]
len(filtered_skeletons) = 1
sk_angles = [142.1879163]
base_final_widths = [204.95339497 198.7870112 ]
overall_ais_width = 189.2934535268635
ais_width = 201.87020308910246
endnodes_to_test = [93]
Skeleton paths distances = [14570.92513435]
Filtered indexes = []
len(filtered_skeletons) = 1
sk_angles = [91.8233016]
Total time for classification = 16.72279167175293


In [101]:
#4) Get the maximum of the axon angles:
all_axon_angles = []
for limb_idx,limb_data in axon_angles.items():
    for candidate_idx,cand_angle in limb_data.items():
        all_axon_angles.append(cand_angle)

if len(axon_angles)>0:
    axon_angle_maximum = np.max(all_axon_angles)
else:
    axon_angle_maximum = 0


if verbose:
    print("\n -- Cell Type Classification Results --")
    print(f"inh_exc_class={inh_exc_class}")
    print(f"spine_category={spine_category}")
    print(f"axon_angles={axon_angles}")
    print(f"n_axons={n_axons}")
    print(f"n_apicals={n_apicals}")
    print(f"neuron_spine_density={neuron_spine_density}")
    print(f"n_branches_processed={n_branches_processed}")
    print(f"skeletal_length_processed={skeletal_length_processed}")
    print(f"n_branches_in_search_radius={n_branches_in_search_radius}")
    print(f"skeletal_length_in_search_radius={skeletal_length_in_search_radius}")

baylor_cell_type_info = dict(
            cell_type_predicted=inh_exc_class,
             spine_category=spine_category,
            axon_angle_maximum = axon_angle_maximum,
             n_axons=n_axons,
             n_apicals=n_apicals,
             spine_density_classifier=neuron_spine_density,
             n_branches_processed=neuron_spine_density,
             skeletal_length_processed=skeletal_length_processed,
             n_branches_in_search_radius=n_branches_in_search_radius,
             skeletal_length_in_search_radius=skeletal_length_in_search_radius,

)



 -- Cell Type Classification Results --
inh_exc_class=excitatory
spine_category=densely_spined
axon_angles={2: {0: 142.18791629917555}}
n_axons=1
n_apicals=0
neuron_spine_density=0.00045453955279439344
n_branches_processed=12
skeletal_length_processed=761645.7600549706
n_branches_in_search_radius=40
skeletal_length_in_search_radius=1310168.901495347


In [100]:
baylor_cell_type_info

{'inh_exc_class': 'excitatory',
 'spine_category': 'densely_spined',
 'axon_angle_maximum': 142.18791629917555,
 'n_axons': 1,
 'n_apicals': 0,
 'spine_density_classifier': 0.00045453955279439344,
 'n_branches_processed': 0.00045453955279439344,
 'skeletal_length_processed': 761645.7600549706,
 'n_branches_in_search_radius': 40,
 'skeletal_length_in_search_radius': 1310168.901495347}

In [86]:
e_i_class = inh_exc_class
if inh_exc_class_to_use_for_axon == "Allen" and allen_cell_type_info["e_i"] is not None:
    e_i_class = allen_cell_type_info["e_i"]

if verbose:
    print(f"e_i_class = {e_i_class} with inh_exc_class_to_use_for_axon = {inh_exc_class_to_use_for_axon}")
    

e_i_class = excitatory with inh_exc_class_to_use_for_axon = Baylor


# Doing the Extra Processing if excitatory

In [91]:
neuron_obj.axon_skeleton

No skeletons to stack so returning empty list


array([], dtype=float64)

In [94]:
au.axon_features_from_neuron_obj(neuron_obj_syn)

{'volume': 41.0166245594561,
 'axon_length': 885.8288170480126,
 'axon_branch_length_median': 21.504822475270704,
 'axon_branch_length_mean': 27.682150532750395,
 'n_branches': 32,
 'n_short_branches': 1,
 'n_long_branches': 0,
 'n_medium_branches': 31,
 'bbox_x_min': 368704.45089587884,
 'bbox_y_min': 822731.5712054871,
 'bbox_z_min': 716488.3858535871,
 'bbox_x_max': 567993.6276426472,
 'bbox_y_max': 961364.2352778121,
 'bbox_z_max': 864948.7889153942,
 'bbox_x_min_soma_relative': -13782.103612991981,
 'bbox_y_min_soma_relative': -67613.74866074487,
 'bbox_z_min_soma_relative': -86621.14802387054,
 'bbox_x_max_soma_relative': 185507.07313377643,
 'bbox_y_max_soma_relative': 71018.91541158012,
 'bbox_z_max_soma_relative': 61839.255037936615}

In [ ]:
nru.labels_to_clear(neuron_obj_syn,labels_to_clear="")

In [95]:
"""
Psuedocode: 
If e_i class is excitatory:
1) Filter away the axon on dendrite
2) Do the higher fidelity axon processing
3) Compute the axon features

"""
axon_features = dict()
if e_i_class == "excitatory" and neuron_obj_syn.axon_limb_name is not None:
    if verbose:
        print(f"Excitatory so performing high fidelity axon and computing axon features")
#     1) Filter away the axon on dendrite
#     2) Do the higher fidelity axon processing
    
    o_neuron, filtering_info = au.complete_axon_processing(neuron_obj_syn,
                                                          perform_axon_classification = False,
                                                          return_filtering_info = True)
    
    #3) Compute the axon features
    axon_features = au.axon_features_from_neuron_obj(o_neuron)


Excitatory so performing high fidelity axon and computing axon features
Recieved another instance of Neuron class in init -- so just copying data
Starting axon branch was the starting node so using border vertices and NOT computing upstream branch
upstream_node_to_axon_starting_branch = None


  0%|          | 0/1 [00:00<?, ?it/s]

root_curr = [385701.7 894389.  799056.6]
Time for preparing soma vertices and root: 0.0005128383636474609
invalidation_d = 1500
smooth_neighborhood = 0
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:01<00:00,  1.75s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 2.52059006690979
connecting at the root
branches_touching_root = [142]
combining close nodes
combine_threshold = 1300
length of Graph = 12169
After combining close endpoints max(kept_branches_idx) = 142, len(kept_branches_idx) = 121
empty_indices % = 0.0
 conflict_indices % = 0.028999659009888712


...


AFTER face_lookup_resolved_test



Limb Correspondence before stitching = 121
Filtering away non soma floating pieces near the soma
Before filter # of pieces = 2
AFTER filter # of pieces = 1





--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (1, 2)
empty_indices % = 0.1695906432748538
 conflict_indices % = 0.011695906432748537



AFTER face_lookup_resolved_test



main_branch = 95
Limb Correspondence AFTER stitching = 123
Recieved another instance of Neuron class in init -- so just copying data
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7f93f33bbba8>]}


----Working on seperate_graph 0----


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [386074.2 893814.6 798823.9]---------
Starting_edge inside branches_to_conept = [[386074.2 893814.6 798823.9]
 [392259.  901976.2 801990. ]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [0]
printing out current edge:
[[386074.2 893814.6 798823.9]
 [392259.  901976.2 801990. ]]
edge_endpoints_to_process was empty so exiting loop after 123 iterations
starting_node in concept map (that should match the starting edge) = 0
Total time for branches to concept conversion = 0.8400771617889404

Done generating concept network 


Recieved another instance of Neuron class in init -- so just copying data


# Table Definition

In [ ]:
n_dict = dict(key,
              split_index = split_index,
              axon_version = au.axon_version,
             decomposition=ret_file_path_str,
             axon_length=neuron_obj_with_web.axon_length,
              run_time = np.round(time.time() - st,2)
             )

In [105]:
allen_cell_type_info

{'e_i': 'excitatory',
 'e_i_n_nuc': 1,
 'cell_type': '5P_IT',
 'cell_type_n_nuc': 1,
 'cell_type_e_i': 'excitatory'}

In [ ]:
o_neuron

In [ ]:
baylor_cell_type_info
allen_cell_type_info
nucleus_info
filtering_info
axon_features

In [104]:
syu.plot_synapses(o_neuron)

Recieved another instance of Neuron class in init -- so just copying data

 Working on visualization type: mesh

 Working on visualization type: skeleton


# How to get the axon stats once made

In [24]:
"""
Things you need to run this: 
axon_skeleton
soa_center

"""

axon_dict = au.axon_stats_from_proof_axon_skeleton(segment_id,
                                              split_index=0,
                                                  verbose = True)
axon_dict

Total Number of Branches = 13
n_short_branches = 2, n_medium_branches = 11, n_long_branches = 0
axon_length = 255.4159272201406, axon_branch_length_median = 11.415776722957128, axon_branch_length_mean = 19.64737901693389
bbox_volume = 3.0413154247717755
bbox_corners = [[370474.39774916 889498.6416864  779683.05042113]
 [449371.19125469 959329.1160665  834885.34357585]]
bbox_corners_soma_relative = [[-12009.60225084   -845.3583136  -23396.94957887]
 [ 66887.19125469  68985.1160665   31805.34357585]]


{'volume': 3.0413154247717755,
 'axon_length': 255.4159272201406,
 'axon_branch_length_median': 11.415776722957128,
 'axon_branch_length_mean': 19.64737901693389,
 'n_branches': 13,
 'n_short_branches': 2,
 'n_long_branches': 0,
 'n_medium_branches': 11,
 'bbox_x_min': 370474.3977491564,
 'bbox_y_min': 889498.6416863955,
 'bbox_z_min': 779683.0504211284,
 'bbox_x_max': 449371.1912546936,
 'bbox_y_max': 959329.1160664978,
 'bbox_z_max': 834885.3435758537,
 'bbox_x_min_soma_relative': -12009.6022508436,
 'bbox_y_min_soma_relative': -845.3583136044908,
 'bbox_z_min_soma_relative': -23396.9495788716,
 'bbox_x_max_soma_relative': 66887.19125469361,
 'bbox_y_max_soma_relative': 68985.11606649775,
 'bbox_z_max_soma_relative': 31805.343575853738}

# How was previously doing the high fidelity preprocessing

In [ ]:
#Run the Axon Decomposition
            neuron_obj_with_web = au.complete_axon_processing(neuron_obj,
                                     verbose=True)